In [2]:
# code for reading json data cosuming less memory
import json
import pandas as pd
def init_ds(json):
    ds= {}
    keys = json.keys()
    for k in keys:
        ds[k]= []
    return ds, keys

def read_json(file):
    dataset = {}
    keys = []
    with open(file) as file_lines:
        for count, line in enumerate(file_lines):
            data = json.loads(line.strip())
            if count ==0:
                dataset, keys = init_ds(data)
            for k in keys:
                dataset[k].append(data[k])
                
        return pd.DataFrame(dataset)

## Executing each read one by one to avoid crash

In [3]:
yelp_review= read_json('/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json')

In [4]:
yelp_review.shape[0]

6990280

In [5]:
yelp_checkin = read_json('/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json')

In [6]:
yelp_tip = read_json('/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json')

In [7]:
yelp_user = read_json('/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json')

## Creating a SQL database and query data

In [8]:
import sqlite3

# Create an in-memory SQLite database
conn = sqlite3.connect('yelp_db')
cursor = conn.cursor()

## Loading each of the dataframes to the database one by one

In [9]:
# Write the DataFrame to a SQLite table
yelp_review.to_sql('reviews', conn, index=False, if_exists='replace')


6990280

In [10]:
yelp_checkin.to_sql('checking', conn, index=False, if_exists='replace')

131930

In [11]:
yelp_tip.to_sql('trip', conn, index=False, if_exists='replace')

908915

In [12]:
yelp_user.to_sql('user', conn, index=False, if_exists='replace')

1987897

### Yelp business file was a tricky one to load. The three json columns needed to be converted into json and then was created as a seperate table

In [13]:
yelp_business = read_json('/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json')

Seperating the json cols

In [14]:
yelp_business_json_cols = yelp_business[['business_id','attributes','hours']].copy()
yelp_business.drop(['attributes','hours'], axis=1, inplace=True)

### Pushing the non json columns for now. Lets deal with the json columns later

In [15]:
yelp_business.to_sql('business', conn, index=False, if_exists='replace')

150346

In [16]:
# handling the json columns 
yelp_business_json_cols.head()

,business_id,attributes,hours
0,Pns2l4eNsfO8kk83dixA6A,{'ByAppointmentOnly': 'True'},None
1,mpf3x-BjTdTEA3yCZrAYPw,{'BusinessAcceptsCreditCards': 'True'},"{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


## Creating a temp table for storing the json columns 

In [17]:
cursor.execute('CREATE TEMP TABLE business_attributes (business_id TEXT, attributes TEXT, hours TEXT)')

In [18]:
# Convert dictionaries to JSON strings
yelp_business_json_cols['hours'] = yelp_business_json_cols['hours'].apply(json.dumps)

In [19]:
yelp_business_json_cols['attributes'] = yelp_business_json_cols['attributes'].apply(json.dumps)

## Inserting all the columns of the json dataframe into the temptable

In [20]:
for row in yelp_business_json_cols.itertuples(index=False):
    cursor.execute('INSERT INTO business_attributes VALUES (?, ?, ?)', row)

In [21]:
query = '''
    SELECT * FROM business LIMIT 3

;'''

result = pd.read_sql_query(query, conn)
result

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,categories
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,"Doctors, Traditional Chinese Medicine, Naturop..."
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,"Shipping Centers, Local Services, Notaries, Ma..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"Department Stores, Shopping, Fashion, Home & G..."


In [22]:
query = '''
    SELECT * FROM business_attributes LIMIT 3

;'''

result = pd.read_sql_query(query, conn)
result

,business_id,attributes,hours
0,Pns2l4eNsfO8kk83dixA6A,"{""ByAppointmentOnly"": ""True""}",null
1,mpf3x-BjTdTEA3yCZrAYPw,"{""BusinessAcceptsCreditCards"": ""True""}","{""Monday"": ""0:0-0:0"", ""Tuesday"": ""8:0-18:30"", ..."
2,tUFrWirKiKi_TAnsVWINQQ,"{""BikeParking"": ""True"", ""BusinessAcceptsCredit...","{""Monday"": ""8:0-22:0"", ""Tuesday"": ""8:0-22:0"", ..."


## We have them sorted.

In [23]:
query = '''
    SELECT business.business_id,name,city,attributes,hours FROM business_attributes JOIN business
    WHERE business_attributes.business_id = business.business_id
    LIMIT 3

;'''

result = pd.read_sql_query(query, conn)
result

,business_id,name,city,attributes,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ",Santa Barbara,"{""ByAppointmentOnly"": ""True""}",null
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,Affton,"{""BusinessAcceptsCreditCards"": ""True""}","{""Monday"": ""0:0-0:0"", ""Tuesday"": ""8:0-18:30"", ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,Tucson,"{""BikeParking"": ""True"", ""BusinessAcceptsCredit...","{""Monday"": ""8:0-22:0"", ""Tuesday"": ""8:0-22:0"", ..."


In [24]:
query = '''

SELECT * FROM business_attributes

;'''

result = pd.read_sql_query(query, conn)
result

,business_id,attributes,hours
0,Pns2l4eNsfO8kk83dixA6A,"{""ByAppointmentOnly"": ""True""}",null
1,mpf3x-BjTdTEA3yCZrAYPw,"{""BusinessAcceptsCreditCards"": ""True""}","{""Monday"": ""0:0-0:0"", ""Tuesday"": ""8:0-18:30"", ..."
2,tUFrWirKiKi_TAnsVWINQQ,"{""BikeParking"": ""True"", ""BusinessAcceptsCredit...","{""Monday"": ""8:0-22:0"", ""Tuesday"": ""8:0-22:0"", ..."
3,MTSW4McQd7CbVtyjqoe9mw,"{""RestaurantsDelivery"": ""False"", ""OutdoorSeati...","{""Monday"": ""7:0-20:0"", ""Tuesday"": ""7:0-20:0"", ..."
4,mWMc6_wTdE0EUBKIGXDVfA,"{""BusinessAcceptsCreditCards"": ""True"", ""Wheelc...","{""Wednesday"": ""14:0-22:0"", ""Thursday"": ""16:0-2..."
...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,"{""ByAppointmentOnly"": ""False"", ""RestaurantsPri...","{""Monday"": ""10:0-19:30"", ""Tuesday"": ""10:0-19:3..."
150342,c8GjPIOTGVmIemT7j5_SyQ,"{""BusinessAcceptsCreditCards"": ""True"", ""Restau...","{""Monday"": ""9:30-17:30"", ""Tuesday"": ""9:30-17:3..."
150343,_QAMST-NrQobXduilWEqSw,"{""RestaurantsPriceRange2"": ""1"", ""BusinessAccep...",null
150344,mtGm22y5c2UHNXDFAjaPNw,"{""BusinessParking"": ""{'garage': False, 'street...","{""Monday"": ""9:0-20:0"", ""Tuesday"": ""9:0-20:0"", ..."


**count of distinct businesses attributes that have a rating of 5.0 stars***

In [25]:
query = '''
   
     SELECT attributes,COUNT(DISTINCT business_attributes.business_id) AS business_count FROM business_attributes JOIN business
    WHERE business_attributes.business_id = business.business_id and business.stars =5.0 and business_attributes.attributes != 'null'  group by attributes
 
;'''

result = pd.read_sql_query(query, conn)
result

,attributes,business_count
0,"{""AcceptsInsurance"": ""False"", ""BikeParking"": ""...",1
1,"{""AcceptsInsurance"": ""False"", ""BikeParking"": ""...",1
2,"{""AcceptsInsurance"": ""False"", ""BikeParking"": ""...",1
3,"{""AcceptsInsurance"": ""False"", ""BikeParking"": ""...",1
4,"{""AcceptsInsurance"": ""False"", ""BusinessAccepts...",1
...,...,...
7844,"{""WiFi"": ""u'paid'"", ""BusinessAcceptsCreditCard...",1
7845,"{""WiFi"": ""u'paid'"", ""ByAppointmentOnly"": ""True...",2
7846,"{""WiFi"": ""u'paid'"", ""DogsAllowed"": ""True"", ""Wh...",1
7847,"{""WiFi"": ""u'paid'"", ""RestaurantsPriceRange2"": ...",1


In [26]:
query = '''
   
     SELECT attributes,COUNT(DISTINCT business_attributes.business_id) AS business_count FROM business_attributes JOIN business
    WHERE business_attributes.business_id = business.business_id and business.stars =5.0 and business_attributes.attributes != 'null'  group by attributes
 
;'''

result = pd.read_sql_query(query, conn)
result

,attributes,business_count
0,"{""AcceptsInsurance"": ""False"", ""BikeParking"": ""...",1
1,"{""AcceptsInsurance"": ""False"", ""BikeParking"": ""...",1
2,"{""AcceptsInsurance"": ""False"", ""BikeParking"": ""...",1
3,"{""AcceptsInsurance"": ""False"", ""BikeParking"": ""...",1
4,"{""AcceptsInsurance"": ""False"", ""BusinessAccepts...",1
...,...,...
7844,"{""WiFi"": ""u'paid'"", ""BusinessAcceptsCreditCard...",1
7845,"{""WiFi"": ""u'paid'"", ""ByAppointmentOnly"": ""True...",2
7846,"{""WiFi"": ""u'paid'"", ""DogsAllowed"": ""True"", ""Wh...",1
7847,"{""WiFi"": ""u'paid'"", ""RestaurantsPriceRange2"": ...",1


**count of reviews for each hour of the day.**

In [27]:

query = '''
SELECT 
    strftime('%H', date) AS review_hour,
    COUNT(review_id) AS review_count
FROM 
    reviews
GROUP BY 
    review_hour
ORDER BY 
    review_count DESC;
    '''

result = pd.read_sql_query(query, conn)
result

,review_hour,review_count
0,18,464488
1,19,463727
2,00,460765
3,01,457189
4,20,452524
5,23,448405
6,17,441397
7,21,439319
8,22,435185
9,02,416472


In [28]:
query = '''
   
   SELECT * from reviews
 
;'''

result = pd.read_sql_query(query, conn)
result

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,KU_O5udG6zpxOg-VcAEodg,mh_-eMZ6K5RLWhZyISBhwA,XQfwVwDr-v0ZS3_CbbE5Xw,3.0,0,0,0,"If you decide to eat here, just be aware it is...",2018-07-07 22:09:11
1,BiTunyQ73aT9WBnpR9DZGw,OyoGAe7OKpv6SyGZT5g77Q,7ATYjTIgM3jUlt4UM3IypQ,5.0,1,0,1,I've taken a lot of spin classes over the year...,2012-01-03 15:28:18
2,saUsX_uimxRlCVr67Z4Jig,8g_iMtfSiwikVnbP2etR0A,YjUWPpI6HXG530lwP-fb2A,3.0,0,0,0,Family diner. Had the buffet. Eclectic assortm...,2014-02-05 20:30:30
3,AqPFMleE6RsU23_auESxiA,_7bHUi9Uuf5__HHc_Q8guQ,kxX2SOes4o-D3ZQBkiMRfA,5.0,1,0,1,"Wow! Yummy, different, delicious. Our favo...",2015-01-04 00:01:03
4,Sx8TMOWLNuJBWer-0pcmoA,bcjbaE6dDog4jkNY91ncLQ,e4Vwtrqf-wpJfwesgvdgxQ,4.0,1,0,1,Cute interior and owner (?) gave us tour of up...,2017-01-14 20:54:15
...,...,...,...,...,...,...,...,...,...
6990275,H0RIamZu0B0Ei0P4aeh3sQ,qskILQ3k0I_qcCMI-k6_QQ,jals67o91gcrD4DC81Vk6w,5.0,1,2,1,Latest addition to services from ICCU is Apple...,2014-12-17 21:45:20
6990276,shTPgbgdwTHSuU67mGCmZQ,Zo0th2m8Ez4gLSbHftiQvg,2vLksaMmSEcGbjI5gywpZA,5.0,2,1,2,"This spot offers a great, affordable east week...",2021-03-31 16:55:10
6990277,YNfNhgZlaaCO5Q_YJR4rEw,mm6E4FbCMwJmb7kPDZ5v2Q,R1khUUxidqfaJmcpmGd4aw,4.0,1,0,0,This Home Depot won me over when I needed to g...,2019-12-30 03:56:30
6990278,i-I4ZOhoX70Nw5H0FwrQUA,YwAMC-jvZ1fvEUum6QkEkw,Rr9kKArrMhSLVE9a53q-aA,5.0,1,0,0,For when I'm feeling like ignoring my calorie-...,2022-01-19 18:59:27


**** businesses that have a star rating of 4.5 or higher ****


In [29]:
query = '''
SELECT b.business_id, b.name AS business_name, b.stars AS business_stars,
       u.user_id, u.name AS user_name, u.review_count AS user_review_count
FROM business b
JOIN reviews r ON b.business_id = r.business_id
JOIN user u ON r.user_id = u.user_id
WHERE b.stars >= 4.5
ORDER BY u.review_count DESC
LIMIT 5;
'''

result = pd.read_sql_query(query,conn)
result

,business_id,business_name,business_stars,user_id,user_name,user_review_count
0,ROeacJQwBeh05Rqg7F6TCg,BAP,4.5,Hi10sGSZNxQH3NLyWSZ1oA,Fox,17473
1,oaboaRBUgGjbo2kfUIKDLQ,Mike's Ice Cream,4.5,Hi10sGSZNxQH3NLyWSZ1oA,Fox,17473
2,sB45WFgysT617bKWP_WJwA,Budweiser Brewery Experience,4.5,Hi10sGSZNxQH3NLyWSZ1oA,Fox,17473
3,2pXrwBssKTp30uxbQgl6kw,St. Pete Beach,4.5,Hi10sGSZNxQH3NLyWSZ1oA,Fox,17473
4,07i5EdI8v2FBhVV-rya8Wg,Pupuseria & Cafeteria Centroamericana,4.5,Hi10sGSZNxQH3NLyWSZ1oA,Fox,17473


**** average review count for each user and returns the top 10 users ****


In [32]:
query = '''
SELECT user_id, AVG(review_count) AS avg_review_count
FROM user
GROUP BY user_id
ORDER BY avg_review_count DESC
LIMIT 10;
'''

result = pd.read_sql_query(query,conn)
result

,user_id,avg_review_count
0,Hi10sGSZNxQH3NLyWSZ1oA,17473.0
1,8k3aO-mPeyhbR5HUucA5aA,16978.0
2,hWDybu_KvYLSdEFzGrniTw,16567.0
3,RtGqdDBvvBCjcu5dUqwfzA,12868.0
4,P5bUL3Engv-2z6kKohB6qQ,9941.0
5,nmdkHL2JKFx55T3nq5VziA,8363.0
6,bQCHF5rn5lMI9c5kEwCaNA,8354.0
7,8RcEwGrFIgkt9WQ35E6SnQ,7738.0
8,Xwnf20FKuikiHcSpcEbpKQ,6766.0
9,CxDOIDnH8gp9KXzpBHJYXw,6679.0
